# Missing data and imputation

## How to recognize missing data?

NaN: not a number
In Pandas: this data is missing

In [1]:
float('NaN')

nan

In [2]:
import pandas as pd

In [3]:
d = pd.read_csv('auto-mpg-orig.csv')

In [4]:
d.describe().round(1)

,mpg,cylinders,displacement,weight,acceleration,year,origin
count,397.0,397.0,397.0,397.0,397.0,397.0,397.0
mean,23.5,5.5,193.5,2970.3,15.6,76.0,1.6
std,7.8,1.7,104.4,847.9,2.7,3.7,0.8
min,9.0,3.0,68.0,1613.0,8.0,70.0,1.0
25%,17.5,4.0,104.0,2223.0,13.8,73.0,1.0
50%,23.0,4.0,146.0,2800.0,15.5,76.0,1.0
75%,29.0,8.0,262.0,3609.0,17.1,79.0,2.0
max,46.6,8.0,455.0,5140.0,24.8,82.0,3.0


397 may be an indicator that no data is missing.
However, if someone replaces the missing data with something, then it is hard to detect.

How to detect it? Let's study the types of the data I can expect in a column!

In [5]:
len(d)

397

In [6]:
d.dtypes

mpg             float64
cylinders         int64
displacement    float64
hp               object
weight          float64
acceleration    float64
year              int64
origin            int64
dtype: object

Don't you notice something strange in the type of "hp"? 
- I didn't notice before, but the describe() function only returns numerical values and in fact hp was not there!

In [7]:
d

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1
...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86.00,2790.0,15.6,82,1
393,44.0,4,97.0,52.00,2130.0,24.6,82,2
394,32.0,4,135.0,84.00,2295.0,11.6,82,1
395,28.0,4,120.0,79.00,2625.0,18.6,82,1


Let's try to force it to consider it as a numeric column

In [8]:
pd.to_numeric(d.hp)

ValueError: Unable to parse string "?" at position 41

What is happening? Are there "?"?

In [9]:
d[d.hp == '?']

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
41,25.0,4,98.0,?,2046.0,19.0,71,1
128,21.0,6,200.0,?,2875.0,17.0,74,1
324,23.6,4,140.0,?,2905.0,14.3,80,1
330,40.9,4,85.0,?,1835.0,17.3,80,2
354,34.5,4,100.0,?,2320.0,15.8,81,2


In [10]:
#pd.to_numeric is going to add a new column, but no changes to d.hp will be done
#whenever pandas raises an error because it does not know how to transform that ? into a nan, just force it and transform it into a nan
d.hp = pd.to_numeric(d.hp, errors='coerce')

In [11]:
d[d.hp == '?']

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin


In [12]:
d.dtypes

mpg             float64
cylinders         int64
displacement    float64
hp              float64
weight          float64
acceleration    float64
year              int64
origin            int64
dtype: object

In [13]:
d.describe().round(1)

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
count,397.0,397.0,397.0,392.0,397.0,397.0,397.0,397.0
mean,23.5,5.5,193.5,104.5,2970.3,15.6,76.0,1.6
std,7.8,1.7,104.4,38.5,847.9,2.7,3.7,0.8
min,9.0,3.0,68.0,46.0,1613.0,8.0,70.0,1.0
25%,17.5,4.0,104.0,75.0,2223.0,13.8,73.0,1.0
50%,23.0,4.0,146.0,93.5,2800.0,15.5,76.0,1.0
75%,29.0,8.0,262.0,126.0,3609.0,17.1,79.0,2.0
max,46.6,8.0,455.0,230.0,5140.0,24.8,82.0,3.0


In [14]:
#now I want to query all rows containing NaN values for hp
d[d.hp.isna()]

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
41,25.0,4,98.0,NaN,2046.0,19.0,71,1
128,21.0,6,200.0,NaN,2875.0,17.0,74,1
324,23.6,4,140.0,NaN,2905.0,14.3,80,1
330,40.9,4,85.0,NaN,1835.0,17.3,80,2
354,34.5,4,100.0,NaN,2320.0,15.8,81,2


In [15]:
d.hp.isna().value_counts()

hp
False    392
True       5
Name: count, dtype: int64

# Data imputation

* **MCAR** Data Missing Completely at Random. Data is missing in a way that is completely unrelated to any other feature in the dataset. The missing data points are essentially random and do not depend on any other variables in the dataset. MCAR implies randomness without relation to any feature. Example: you have a dataset with temperature readings from various sensors. If some sensors fail randomly due to external factors (e.g., power outages) that are not related to temperature or any other variable in your dataset, this would be considered MCAR.
* **MAR** Data Missing at Random. Data is missing in a way that is related to other observed features but not to the value of the feature itself that is missing. By analyzing other features in the dataset, one can predict which values are likely to be missing. MAR implies dependency on other observed features but not on the feature with missing values itself. Example: Temperature readings are more likely to be missed when humidity levels are high (missingness explained by another feature)
* **MNAR** Data Missing Not at Random. Data is missing in a way that depends on the value of the feature itself that is missing. Missingness cannot be fully explained by other observed variables; it depends on information we do not have because it’s part of what’s missing. MNAR implies dependency on the feature with its own missing values. Example: High temperatures cause sensor failures leading directly to those high-temperature readings being missed.

Simple ways of doing imputation:
* Impute the feature mean
* Impute the feature median
* Impute the mean/median of a subset of rows

More advanced ways of doing imputation:
* MICE: Multiple Imputation by Chained Equations
* Imputation by nearest neighbours (e.g., using the $k$-NN algorithm)

In [16]:
#Impute with the feature mean (By running the below command, for our example, it will impute the mean only for the NaN values in hp. However, if we had also NaN values for other columns, then they would be imputed with the mean of that column too)

d_imp1 = d.fillna(d.mean())

In [17]:
d_imp1.describe().round(1)

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
count,397.0,397.0,397.0,397.0,397.0,397.0,397.0,397.0
mean,23.5,5.5,193.5,104.5,2970.3,15.6,76.0,1.6
std,7.8,1.7,104.4,38.2,847.9,2.7,3.7,0.8
min,9.0,3.0,68.0,46.0,1613.0,8.0,70.0,1.0
25%,17.5,4.0,104.0,76.0,2223.0,13.8,73.0,1.0
50%,23.0,4.0,146.0,95.0,2800.0,15.5,76.0,1.0
75%,29.0,8.0,262.0,125.0,3609.0,17.1,79.0,2.0
max,46.6,8.0,455.0,230.0,5140.0,24.8,82.0,3.0


In [18]:
d_imp1[d.hp.isna()] #show me the data that in the old dataset "d" had not a number

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
41,25.0,4,98.0,104.469388,2046.0,19.0,71,1
128,21.0,6,200.0,104.469388,2875.0,17.0,74,1
324,23.6,4,140.0,104.469388,2905.0,14.3,80,1
330,40.9,4,85.0,104.469388,1835.0,17.3,80,2
354,34.5,4,100.0,104.469388,2320.0,15.8,81,2


In [19]:
#impute with median
d_imp2 = d.fillna(d.median())

In [20]:
d_imp2[d.hp.isna()]

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
41,25.0,4,98.0,93.5,2046.0,19.0,71,1
128,21.0,6,200.0,93.5,2875.0,17.0,74,1
324,23.6,4,140.0,93.5,2905.0,14.3,80,1
330,40.9,4,85.0,93.5,1835.0,17.3,80,2
354,34.5,4,100.0,93.5,2320.0,15.8,81,2


In [21]:
#impute with the mean of a group 
#make sense if you have correlated features (e.g., cylinders and hp: I want to impute the mean for those rows with only 4 cylinders)

d.corr().round(2)

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
mpg,1.00,-0.78,-0.80,-0.78,-0.83,0.42,0.58,0.56
cylinders,-0.78,1.00,0.95,0.84,0.90,-0.50,-0.35,-0.56
displacement,-0.80,0.95,1.00,0.90,0.93,-0.54,-0.37,-0.61
hp,-0.78,0.84,0.90,1.00,0.86,-0.69,-0.42,-0.46
weight,-0.83,0.90,0.93,0.86,1.00,-0.42,-0.31,-0.58
acceleration,0.42,-0.50,-0.54,-0.69,-0.42,1.00,0.28,0.21
year,0.58,-0.35,-0.37,-0.42,-0.31,0.28,1.00,0.18
origin,0.56,-0.56,-0.61,-0.46,-0.58,0.21,0.18,1.00


In [22]:
d_imp3 = d.copy()

In [24]:
#groupby: group dataset by cylinder (e.g., all datapoints are grouped into groups of 2, 3, 4, ... cilinders)  
#transform: I can apply any action to any of the groups created with the groupby
#lambda (keyword): requires a variable (group). What do I want to do within each group
#Note: if you don't put ".hp" at the end, you will do that for all columns
d_imp3.hp = d.groupby('cylinders').transform(lambda group: group.fillna(group.mean())).hp

In [25]:
d_imp3[d.hp.isna()]

,mpg,cylinders,displacement,hp,weight,acceleration,year,origin
41,25.0,4,98.0,78.281407,2046.0,19.0,71,1
128,21.0,6,200.0,101.506024,2875.0,17.0,74,1
324,23.6,4,140.0,78.281407,2905.0,14.3,80,1
330,40.9,4,85.0,78.281407,1835.0,17.3,80,2
354,34.5,4,100.0,78.281407,2320.0,15.8,81,2


In [26]:
#just to check that it worked as expected:
d[d.cylinders == 4].hp.mean()

78.28140703517587

In [27]:
d[d.cylinders == 6].hp.mean()

101.50602409638554

## How to impute a categorical variable?

Most common method: impute the mode (the value that is occurring most often)
- on the complete dataset
- on grouped datapoints